In [25]:
# 1. Добавить временные ряды
# 2. Grid Search с разными энкодерами и скейлерами
# 3. Optuna
# 4. Блендирнг или стейкинг
# 5. Tabnet
# 6. Feature engineering

# Import

In [26]:
# !pip install pyarrow ipywidgets

In [27]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score, classification_report, make_scorer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_validate, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from catboost import CatBoostClassifier
pd.set_option('display.max_rows', 500)
random_state = 42
data_filepath = './data/'
data_filepath = '/kaggle/input/child-mind-institute-problematic-internet-use/'

# Data

In [28]:
!du -hs $data_filepath*

12K	/kaggle/input/child-mind-institute-problematic-internet-use/data_dictionary.csv
4.0K	/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv
8.0M	/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet
6.3G	/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet
8.0K	/kaggle/input/child-mind-institute-problematic-internet-use/test.csv
924K	/kaggle/input/child-mind-institute-problematic-internet-use/train.csv


In [29]:
train_df = pd.read_csv(data_filepath + 'train.csv')
test_df = pd.read_csv(data_filepath + 'test.csv')
train_df.shape, test_df.shape

((3960, 82), (20, 59))

In [30]:
target_source_cols = [c for c in train_df.columns if c.startswith('PCIAT')]
train_df.drop(columns=target_source_cols, inplace=True)
train_df.shape

(3960, 60)

In [31]:
train_df.sii.value_counts(dropna=False)

sii
0.0    1594
NaN    1224
1.0     730
2.0     378
3.0      34
Name: count, dtype: int64

In [32]:
train_df = train_df.dropna(subset='sii')
train_df.shape

(2736, 60)

# Cat num cols split 

In [33]:
cols_to_drop = ['sii', 'id']
X, y = train_df.drop(columns=cols_to_drop), train_df.sii
X.shape, y.shape

((2736, 58), (2736,))

In [34]:
cat_cols = X.select_dtypes(include='object').columns.tolist()
num_cols = X.select_dtypes(include='number').columns.tolist()

# Pipeline

In [35]:
numeric_transormer = Pipeline(steps=[
    # ('imputer', SimpleImputer(strategy='median')),
    ('imputer', SimpleImputer(strategy='constant', fill_value=-999)),
])  
category_transormer = Pipeline(steps=[
    # ('imputer', SimpleImputer(strategy='most_frequent')),
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OrdinalEncoder()),
])
cols_transformer = ColumnTransformer(transformers=[
    ('numeric', numeric_transormer, num_cols),
    ('category', category_transormer, cat_cols),
])
preproc_df_pipe = Pipeline(steps=[
    ('cols_transformer', cols_transformer),
    ('scaler', StandardScaler()),
])
preproc_df_pipe

Pipeline(steps=[('cols_transformer',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=-999,
                                                                                 strategy='constant'))]),
                                                  ['Basic_Demos-Age',
                                                   'Basic_Demos-Sex',
                                                   'CGAS-CGAS_Score',
                                                   'Physical-BMI',
                                                   'Physical-Height',
                                                   'Physical-Weight',
                                                   'Physical-Waist_Circumference',
                                                   'Physical-Diastolic_BP',
                                                   'Physical-HeartRate',
                                                   'Physica...
                                                   'BIA-BIA_BMC', 'BIA-BIA_BMI', ...]),
                                                 ('category',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OrdinalEncoder())]),
                                                  ['Basic_Demos-Enroll_Season',
                                                   'CGAS-Season',
                                                   'Physical-Season',
                                                   'Fitness_Endurance-Season',
                                                   'FGC-Season', 'BIA-Season',
                                                   'PAQ_A-Season',
                                                   'PAQ_C-Season', 'SDS-Season',
                                                   'PreInt_EduHx-Season'])])),
                ('scaler', StandardScaler())])

# Train 

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, stratify=y, random_state=random_state)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2188, 58), (548, 58), (2188,), (548,))

In [37]:
X_train = preproc_df_pipe.fit_transform(X_train)
X_test = preproc_df_pipe.transform(X_test)
X_train.shape, X_test.shape

((2188, 58), (548, 58))

In [38]:
%%time
clf = CatBoostClassifier()
clf.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    logging_level='Verbose', 
)

Learning rate set to 0.10968
0:	learn: 1.3032797	test: 1.3043475	best: 1.3043475 (0)	total: 21.3ms	remaining: 21.3s
1:	learn: 1.2397225	test: 1.2428041	best: 1.2428041 (1)	total: 35.2ms	remaining: 17.6s
2:	learn: 1.1867615	test: 1.1890074	best: 1.1890074 (2)	total: 48.7ms	remaining: 16.2s
3:	learn: 1.1417180	test: 1.1456159	best: 1.1456159 (3)	total: 61.8ms	remaining: 15.4s
4:	learn: 1.1055714	test: 1.1103992	best: 1.1103992 (4)	total: 74.7ms	remaining: 14.9s
5:	learn: 1.0780245	test: 1.0822822	best: 1.0822822 (5)	total: 89.5ms	remaining: 14.8s
6:	learn: 1.0504643	test: 1.0563172	best: 1.0563172 (6)	total: 103ms	remaining: 14.6s
7:	learn: 1.0265240	test: 1.0345169	best: 1.0345169 (7)	total: 116ms	remaining: 14.3s
8:	learn: 1.0066226	test: 1.0154191	best: 1.0154191 (8)	total: 129ms	remaining: 14.3s
9:	learn: 0.9896852	test: 0.9998913	best: 0.9998913 (9)	total: 143ms	remaining: 14.2s
10:	learn: 0.9731568	test: 0.9860574	best: 0.9860574 (10)	total: 157ms	remaining: 14.1s
11:	learn: 0.9572

In [39]:
preds = clf.predict(X_test)
print(cohen_kappa_score(y_test, preds))
print(classification_report(y_test, preds))

0.24206229312278016
              precision    recall  f1-score   support

         0.0       0.68      0.91      0.78       319
         1.0       0.46      0.29      0.35       146
         2.0       0.32      0.13      0.19        76
         3.0       0.00      0.00      0.00         7

    accuracy                           0.62       548
   macro avg       0.37      0.33      0.33       548
weighted avg       0.56      0.62      0.57       548



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
fi = clf.get_feature_importance(prettified=True)
fi['Feature name'] = fi['Feature Id'].apply(lambda i: X.columns[int(i)])
fi

,Feature Id,Importances,Feature name
0,47,9.531135,BIA-BIA_SMM
1,46,5.797507,BIA-BIA_LST
2,45,5.354437,BIA-BIA_LDM
3,0,5.316825,Basic_Demos-Enroll_Season
4,1,3.607892,Basic_Demos-Age
5,5,3.503148,Physical-Season
6,8,3.180329,Physical-Weight
7,9,3.159010,Physical-Waist_Circumference
8,4,2.881777,CGAS-CGAS_Score
9,49,2.754726,PAQ_A-Season


# Cross validation

In [41]:
%%time
_clf = CatBoostClassifier(verbose=0)
cv = StratifiedShuffleSplit(n_splits=5, test_size=.3, random_state=random_state)
_pipeline = Pipeline([('transformer', preproc_df_pipe), ('estimator', _clf)])
scoring = {
    'f1': 'f1_macro',
    'cohen_kappa_score': make_scorer(cohen_kappa_score), 
}
scores = cross_validate(_pipeline, X, y, cv=cv, scoring=scoring)
for metric, folds_score in scores.items():
    if not metric.startswith('test_'): continue
    print(metric, folds_score.mean().round(4), folds_score.std().round(4))

test_f1 0.3114 0.0043
test_cohen_kappa_score 0.1738 0.0149
CPU times: user 3min 52s, sys: 13.2 s, total: 4min 6s
Wall time: 1min 6s


# Save

In [42]:
test_df = pd.read_csv(data_filepath + 'test.csv')
test_df['sii'] = clf.predict(preproc_df_pipe.transform(test_df[X.columns]))
test_df.sii.value_counts()

sii
0.0    19
1.0     1
Name: count, dtype: int64

In [43]:
test_df[['id', 'sii']].to_csv('submission.csv', index=False)
!du -hs 'submission.csv'

4.0K	submission.csv


In [ ]:
# Score: 0.307